In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!cp /content/drive/"My Drive"/kaggle/lish-moa.zip /content/

In [2]:
!unzip lish-moa.zip

Archive:  lish-moa.zip
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [3]:
!pip install pip install iterative-stratification

In [ ]:
!pip install optuna

In [5]:
from sklearn.preprocessing import MinMaxScaler

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler

import optuna
import numpy as np 
import pandas as pd 
from IPython.display import clear_output
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=42):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    
seed_everything(seed=42)

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [6]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device


'cuda'

In [7]:
train_features = pd.read_csv('train_features.csv')
train_targets = pd.read_csv('train_targets_scored.csv')
train_targets_s = train_targets
test_features = pd.read_csv('test_features.csv')

ss = pd.read_csv('sample_submission.csv')

In [8]:
def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

target = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [9]:
top_features = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]

all_columns = train.columns
train=train[all_columns[top_features]]
test = test[all_columns[top_features]]
train.shape, test.shape

((21948, 785), (3982, 785))

In [10]:
train = train.values
target = target.values
test = test.values

In [11]:
train.shape, target.shape

((21948, 785), (21948, 206))

In [12]:
class TrainDataset(Dataset):
    def __init__(self, train,targets, noise ):
        
        self.features  = train
        self.targets = targets
        self.noise = noise
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx]).float()
        target = torch.tensor(self.targets[idx]).float()
        return feature, target


In [13]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']
        
def show_lr(learning_rates):
    plt.plot(learning_rates, label = "learning rate")
    plt.ylabel("Learning rate", fontsize = 15)
    plt.grid(True)
    plt.legend()
    plt.show()

def train_step(x, y, model, optimizer, criterion):
    optimizer.zero_grad()
    pred = model(x.to(device))
    y = y.float()
    loss = criterion(pred,y.to(device))
    loss.backward()
    optimizer.step()
    return loss.item()

In [14]:
class Model(nn.Module):
  def __init__(self, nfeatures, ntargets, nlayers, hidden_size, dropout):
    super().__init__()
    layers = []
    for _ in range(nlayers):
      if len(layers) == 0:
        layers.append(nn.Linear(nfeatures, hidden_size))
        layers.append(nn.BatchNorm1d(hidden_size))
        layers.append(nn.Dropout(dropout))
        layers.append(nn.LeakyReLU())
      else:
        layers.append(nn.Linear(hidden_size, hidden_size))
        layers.append(nn.BatchNorm1d(hidden_size))
        layers.append(nn.Dropout(dropout))
        layers.append(nn.LeakyReLU())

    layers.append(nn.Linear(hidden_size, ntargets))

    self.model = nn.Sequential(*layers)

  def forward(self, x):
    return self.model(x)

In [21]:
def run_training( params, save_model=False):
  NFOLDS = 1
  EPOCHS = 20 ## changes here 

  fold_val_losses = list()
      
  full_dataset = TrainDataset(train, target, noise = False)
    
  train_size = int(0.9 * len(full_dataset))  ## 90/10 split
  test_size = len(full_dataset) - train_size
  train_dataset, val_dataset = random_split(full_dataset, [train_size, test_size])

  train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True, num_workers = 8)
  val_loader = DataLoader(dataset=val_dataset, batch_size=256, shuffle = True, num_workers = 8)
  

  model = Model(nfeatures=train.shape[1], 
                ntargets= target.shape[1],
                nlayers=params["num_layers"], 
                hidden_size=params["hidden_size"], 
                dropout=params["dropout"])
  
  model = model.cuda()
  optimizer = optim.Adam(model.parameters(), lr = params["learning_rate"], weight_decay=1e-5)
  scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                  mode='min', 
                                                  factor=0.5, 
                                                  patience=3, 
                                                  eps=1e-4, 
                                                  verbose=True)
  criterion = nn.BCEWithLogitsLoss()

  eng = Engine(model, optimizer, device='cuda')
  best_loss = 10000


  for epoch in range(EPOCHS):
    train_loss = eng.train(train_loader)
    valid_loss = eng.evaluate(val_loader)
    print("train_loss:", train_loss, "val_loss:", valid_loss)
    if valid_loss<best_loss:
      best_loss = valid_loss
      if save_model:
        torch.save(model.state_dict(), "model_{fold}.pth")
    
  return(best_loss)

  print(CBLUE, "Training complete", CEND)

In [22]:
class Engine:
  def __init__(self, model, optimizer, device):
    self.model = model
    self.device = device
    self.optimizer = optimizer

  @staticmethod
  def loss_fn(targets, outputs):
    return nn.BCEWithLogitsLoss()(outputs, targets)

  def train(self, data_loader):
    self.model.train()
    final_loss=0
    for data in data_loader:
      self.optimizer.zero_grad()
      inputs, targets = data
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = self.model(inputs)
      loss = self.loss_fn(targets, outputs)
      loss.backward()
      self.optimizer.step()
      final_loss += loss.item()
    return(final_loss / len(data_loader))


  def evaluate(self, data_loader):
    self.model.train()
    final_loss=0
    for data in data_loader:
      #self.optimizer.zero_grad()
      inputs, targets = data
      inputs, targets = inputs.to(device), targets.to(device)
      outputs = self.model(inputs)
      loss = self.loss_fn(targets, outputs)
      #loss.backward()
      #self.optimizer.step()
      final_loss += loss.item()
    return(final_loss / len(data_loader))

In [23]:
# params = {
#       "num_layers":trial.suggest_int("num_layer", 1, 8),
#       "hidden_size":trial.suggest_int("hidden_size", 16, 4096),
#       "dropout": trial.suggest_uniform("dropout", 0.1, 0.7),
#       "learning_rate": trial.suggest_loguniform("learning_rate", 1e-6, 1e-2)
#   }
# run_training(params, save_model=False)

### Note :
    Replace trial.suggest part with a constant if optimization in not necessary

In [24]:
def objective(trial):
  params = {
      "num_layers":trial.suggest_int("num_layer", 2, 7),
      "hidden_size":trial.suggest_int("hidden_size", 512, 2048),
      "dropout": trial.suggest_uniform("dropout", 0.2, 0.7),
      "learning_rate": trial.suggest_loguniform("learning_rate", 1e-4, 5e-3)
  }

  loss_ = run_training(params, save_model=False)
  return(loss_)

In [25]:
study = optuna.create_study(direction="minimize")

[I 2020-09-26 08:26:19,219] A new study created in memory with name: no-name-ee40455a-289f-4a45-9165-481b76409190


In [ ]:
study.optimize(objective, n_trials=100)

print("best_trial:")
trial_ = study.best_trial
print(trial_)